In [2]:
import numpy as np
import random as rd
from numba import njit

@njit()
def Calculate(arr_card_player):
   arr_point = np.where(arr_card_player == 1)[0]
   point_player = 0
   for idx in arr_point:
      if idx%13 >= 9:
         point_player += 10
      if idx%13 < 9:
         point_player += (idx%13 + 1)
   return point_player

@njit()
def initEnv():
   env_state = np.full(271,0)
   env_state[:52] = 1
   env_state[53:57] = 1

   temp = np.arange(52)
   np.random.shuffle(temp)
   env_state[0+temp[:28]] = -1
   env_state[57+temp[:7]] = 1
   env_state[109+temp[7:14]] = 1
   env_state[161+temp[14:21]] = 1
   env_state[213+temp[21:28]] = 1
   
   for i_ in range(4):
      arr_card_player = env_state[(57+i_*52):(57+(i_+1)*52)]
      point_player = Calculate(arr_card_player)
      env_state[267+i_] = point_player

   return env_state

In [3]:
@njit()
def getAgentState(env_state):
   P_state = np.full(110,0)
   Player_ID = env_state[53]
   P_state[:52] = env_state[(57+Player_ID*52):(57+(Player_ID+1)*52)]
   P_state[52:104] = env_state[:52]
   for idx in range(4):
      if idx+Player_ID <= 3:
         P_state[104+idx] = env_state[53+idx+Player_ID]
      if idx+Player_ID > 3:
         P_state[104+idx] = env_state[49+idx+Player_ID]
   
   check_point = np.where(P_state[:52] == 1)[0]
   point = 0
   for idx in check_point:
      if idx%13 >= 9:
         point += 10
      if idx%13 < 9:
         point += (idx%13 + 1)
   P_state[108] = point
   P_state[109] = env_state[266]

   return P_state.astype(np.float64)

In [4]:
@njit()
def straight_subsequences(arr):
   arr_return = []
   n = len(arr)
   for k in range(3,14):
      if n < k:
         arr_return = np.full((0,2),0)
         return arr_return
      
      for i in range(0, n-k+1):
         sub_arr = arr[i:i+k]
         if np.max(sub_arr) - np.min(sub_arr) == k-1:
            arr_return.append([sub_arr[0], sub_arr[k-1]])
   if len(arr_return) == 0:
      arr_return = np.full((0,2),0)
      return arr_return
   else:
      return np.array(arr_return)

In [5]:
@njit()
def breed_set(arr):
   arr_return_breed = []
   n = len(arr)
   for i_ in range(n):
      numbers_of_card_breed = 0
      for k_ in range(i_,n):
         if (arr[k_] - arr[i_]) % 13 == 0:
            numbers_of_card_breed += 1
         if numbers_of_card_breed >= 3 and (arr[k_] - arr[i_])%13==0:
            arr_return_breed.append([numbers_of_card_breed, arr[i_]])
   if len(arr_return_breed) == 0:
      arr_return_breed = np.full((0,2),0)
      return arr_return_breed
   else:
      return np.array(arr_return_breed)

In [6]:
@njit()
def pick_card_peeps(arr_2):
   arr_pick_peeps = np.where(arr_2 == 1)[0]
   return arr_pick_peeps

In [7]:
@njit()
def pick_card_upside_dow(arr_2):
   arr_pick_upside_dow = np.where(arr_2 == 0)[0]
   return arr_pick_upside_dow

In [8]:
@njit()
def Pay_card(arr_3):
   arr_pay_card = np.where(arr_3 == 1)[0]
   return arr_pay_card

In [9]:
@njit()
def getValidActions(P_state):
   if P_state[109] == 0:
      arr_2 = P_state[52:104]
      arr_pick_peeps = pick_card_peeps(arr_2)
      arr_pick_upside_dow = pick_card_upside_dow(arr_2)

      arr = np.where(P_state[:52] == 1)[0]   #On hand
      arr_straight_subsequences = straight_subsequences(arr)
      arr_breed_set = breed_set(arr)
      n = len(arr_pick_peeps)+len(arr_pick_upside_dow)+len(arr_straight_subsequences)+len(arr_breed_set)
      Valid_Actions_return = np.full(n,1)

      return Valid_Actions_return.astype(np.int64)
   if P_state[109] == 1:
      arr_3 = P_state[:52]
      arr_pay_card = Pay_card(arr_3)
      Valid_Actions_return = np.full(len(arr_pay_card),1)

      return Valid_Actions_return.astype(np.int64)
      

In [10]:
@njit()
def check_status_player(arr_status):
   arr_player = np.where(arr_status == 1)[0]
   Numbers_of_people_player = len(arr_player)
   return Numbers_of_people_player

In [11]:
@njit()
def stepEnv(action,env_state):
   Player_ID = env_state[53]
   if env_state[52] == 0:
      check_winner_begin = np.where(env_state[267:271] == 50)[0]
      if len(check_winner_begin) >= 1:
         winner = check_winner_begin[0]
      for k in range(4):
         if k != winner:
            env_state[53+k] = 0
   arr_status = env_state[53:57]
   Numbers_of_people_player = check_status_player(arr_status)
   if Numbers_of_people_player >= 2:
      if env_state[266] == 0:
         check_arr = env_state[(57+Player_ID*52):(57+(Player_ID+1)*52)]
         arr = np.where(check_arr == 1)[0]   #On Hand
         arr_straight_subsequences = straight_subsequences(arr)
         arr_breed_set = breed_set(arr)

         arr_2 = env_state[:52]
         arr_pick_peeps = pick_card_peeps(arr_2)
         arr_pick_upside_dow = pick_card_upside_dow(arr_2)

         if action < len(arr_pick_peeps): #úp
            card_choose = arr_pick_peeps[action]
            arr_2[card_choose] = -1
            env_state[:52] = arr_2
            env_state[265] = card_choose
            env_state[266] = 1
         if len(arr_pick_peeps) <= action < (len(arr_pick_peeps)+len(arr_pick_upside_dow)):
            place = action - len(arr_pick_peeps)
            card_choose = arr_pick_peeps[place]
            arr_2[card_choose] = -1
            env_state[:52] = arr_2
            env_state[265] = card_choose
            env_state[266] = 1
         if (len(arr_pick_peeps)+len(arr_pick_upside_dow)) <= action < (len(arr_pick_peeps)+len(arr_pick_upside_dow)+len(arr_straight_subsequences)):
            place = action - len(arr_straight_subsequences)
            selected_deck = arr_straight_subsequences[place]
            check_arr[(0+selected_deck[0]):(selected_deck[1]+1)] = 0
            env_state[(57+Player_ID*52):(57+(Player_ID+1)*52)] = check_arr
            env_state[(0+selected_deck[0]):(selected_deck[1]+1)] = 0
            env_state[52] += 1
         if (len(arr_pick_peeps)+len(arr_pick_upside_dow)+len(arr_straight_subsequences)) <= action < len(arr_pick_peeps)+len(arr_pick_upside_dow)+len(arr_straight_subsequences) + len(arr_breed_set):
            place = action - len(arr_breed_set)
            selected_deck = arr_breed_set[place]
            for i_ in range(len(selected_deck[0])):
               check_arr[selected_deck[1]+i_%13] = 0
               env_state[(57+Player_ID*52):(57+(Player_ID+1)*52)] = check_arr
               env_state[selected_deck[1]+i_%13] = 0
            env_state[52] += 1
      if env_state[266] == 1:
         arr_3 = env_state[(57+Player_ID*52):(57+(Player_ID+1)*52)]
         arr_pay_card = Pay_card(arr_3)
         card_pay = arr_pay_card[action]
         arr_3[card_pay] = 0
         env_state[(57+Player_ID*52):(57+(Player_ID+1)*52)] = arr_3
         env_state[card_pay+0] = 0
         env_state[57+Player_ID*52+env_state[265]] = 1
         env_state[265:267] = 0
         env_state[52] += 1
      ###____________________________________________Reset_điểm_cho_player__________________________________________________###

      arr_card_player = env_state[(57+Player_ID*52):(57+(Player_ID+1)*52)]
      point_player = Calculate(arr_card_player)
      env_state[267+Player_ID] = point_player

      ###___________________________________________Check_out_điểm_________________________________________________________###

      if env_state[267+Player_ID] >= 100:
         env_state[53+Player_ID] = 0
      ###___________________________________________check_len(card)==0______________________________________________________###

      check_len = np.where(arr_card_player == 1)[0]
      if len(check_len) == 0:
         env_state[53+Player_ID] = 0
      ###_________________________________________người nhận action tiếp theo_____________________________________________###

      arr_the_following_status = env_state[53:57]
      the_following_status = np.where(arr_the_following_status == 1)[0]
      if len(the_following_status) >= 2:
         if env_state[266] == 1:
            env_state[53] += 0
         if env_state[266] == 0:
            next_player = np.where(the_following_status > env_state[53])[0]
            if len(next_player) == 0:
               env_state[53] = the_following_status[0]
            if len(next_player) > 0:
               place = next_player[0]
               env_state[53] = the_following_status[place]

   return env_state

In [12]:
@njit()
def getAgentsize():
   return 4

In [13]:
@njit()
def checkEnded(env_state):
   arr_status = env_state[53:57]
   check_win_player = np.where(arr_status == 1)[0]
   if len(check_win_player) == 1:
      return check_win_player[0]
   else:
      return -1

In [14]:
@njit()
def getReward(P_state):
   arr_individual_score_win = P_state[104:108]
   individual_score_win = np.where(arr_individual_score_win == 1)[0]
   if len(individual_score_win) >= 2:
      return 0
   if len(individual_score_win) == 1:
      if individual_score_win[0] == 0:
         return 1
      if individual_score_win[0] != 0:
         return -1

In [15]:
@njit()
def getStateSize():
   return 110

In [16]:
def run(listAgent, perData):
   env_state = initEnv()
   tempData = [[0],[0],[0],[0]]

   winner = -1
   Id_player = env_state[53]
   while env_state[52] < 400:
      pIdx = env_state[53]
      P_state = getAgentState(env_state)
      list_action = getValidActions(P_state)
      try:
         action, tempData[pIdx], perData = listAgent[pIdx](P_state, tempData[pIdx], perData)
      except:
         print(list(env_state))
      
      if list_action[action] != 1:
         raise Exception('Người chơi trả về action lỗi')

      stepEnv(action,env_state)
      winner = checkEnded(env_state)
      if winner != -1:
         break
   
   for pIdx in range(4):
      Id_player = pIdx
      P_state = getAgentState(env_state)
      action, tempData[pIdx], perData = listAgent[pIdx](P_state, tempData[pIdx], perData)

   return winner, perData

In [17]:
def main(listAgent, times, perData):
   if len(listAgent) != 4:
      print('Hệ thống cho đúng 4 người chơi:>>>>')

   numWin = np.full(5,0)
   pIdOrder = np.arange(4)
   for w_ in range(times):
      np.random.shuffle(pIdOrder)
      winner, perData = run([listAgent[pIdOrder[0]], listAgent[pIdOrder[1]], listAgent[pIdOrder[2]], listAgent[pIdOrder[3]]], perData)

      if winner == -1:
         numWin[4] += 1
      else:
         numWin[pIdOrder[winner]] += 1
   return numWin, perData

In [18]:
def random_player(P_state, tempData, perData):
   ValidActions = getValidActions(P_state)
   ValidActions = np.where(ValidActions == 1)[0]
   print(ValidActions)
   action = np.random.choice(ValidActions)
   print(action)
   return action, tempData, perData

In [ ]:
def visualize(env_state):
   dictionary = {}
   dictionary["Lá bài trên tay người thứ nhất"] = np.where(env_state[57:109] == 1)[0]
   dictionary["Lá bài trên tay người thứ hai"] = np.where(env_state[109:161] == 1)[0]
   dictionary["Lá bài trên tay người thứ ba"] = np.where(env_state[161:213] == 1)[0]
   dictionary["Lá bài trên tay người thứ ba"] = np.where(env_state[213:265] == 1)[0]
   dictionary["Trạng thái của các người chơi"] = env_state[53:57]
   dictionary["Điểm của người chơi"] = env_state[267:271]
   dictionary["Ai đang action"] = env_state[53]

   return dictionary

In [58]:
env_state = initEnv()
P_state = getAgentState(env_state)
# list_action = getValidActions(P_state)
arr = np.where(P_state[:52] == 1)[0]
print(arr)
arr_return_breed = breed_set(arr)
arr_return = straight_subsequences(arr)
print(len(arr_return_breed))
print(arr_return)


[14 16 26 33 40 43 44]
0
[]
